# Hand Gestures Capstone Project Prototype LSTM ML Model

The model below uses LSTM ML Model to learn patterns of normal, tremor, tonic, and postural phases in a epileptic seizure. The training CSV files are organized in a directory:<br>
data/<br>
├── normal/<br>
│   ├── normal1.csv<br>
│   └── normal2.csv<br>
├── tremor/<br>
│   ├── tremor1.csv<br>
│   └── tremor2.csv<br>
...<br>

And the files will be fed into the ML model to to final result of 4 categorical outputs of normal, tremor, tonic, and postural phases with varying probability.

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class MotionDataset(Dataset):
    def __init__(self, csv_file, sequence_length=100):
        self.data = pd.read_csv(csv_file)
        self.sequence_length = sequence_length

        # Normalize or standardize features if needed
        self.features = self.data[[
            'AccelX(g)', 'AccelY(g)', 'AccelZ(g)',
            'GyroX(deg/s)', 'GyroY(deg/s)', 'GyroZ(deg/s)',
            'DistanceLeft(cm)', 'DistanceRight(cm)'
        ]].values

        # Placeholder: Assume labels are in a column called "label"
        self.labels = self.data['label'].values

        # Chunk into sequences
        self.sequences, self.sequence_labels = self.create_sequences()

    def create_sequences(self):
        sequences = []
        labels = []
        for i in range(0, len(self.features) - self.sequence_length):
            sequence = self.features[i:i+self.sequence_length]
            label = self.labels[i+self.sequence_length-1]  # Label at end of window
            sequences.append(sequence)
            labels.append(label)
        return np.array(sequences), np.array(labels)

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        return torch.tensor(self.sequences[idx], dtype=torch.float32), torch.tensor(self.sequence_labels[idx], dtype=torch.long)


In [ ]:
import os
from glob import glob

def get_file_label_pairs_by_folder(base_dir):
    label_map = {}
    file_label_pairs = []

    for i, folder_name in enumerate(sorted(os.listdir(base_dir))):
        folder_path = os.path.join(base_dir, folder_name)
        if os.path.isdir(folder_path):
            label_map[folder_name] = i
            for file_path in glob(os.path.join(folder_path, "*.csv")):
                file_label_pairs.append((file_path, i))
    
    return file_label_pairs, label_map

# Example:
file_label_pairs, label_map = get_file_label_pairs_by_folder("data")
print("Labels:", label_map)


In [ ]:
import torch.nn as nn

class SeizureLSTM(nn.Module):
    def __init__(self, input_size=8, hidden_size=64, num_layers=2, num_classes=4):
        super(SeizureLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = out[:, -1, :]  # Last time step
        out = self.fc(out)
        return out

In [ ]:
from torch.utils.data import DataLoader

def train(model, dataset, epochs=20, batch_size=32, lr=0.001):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for X_batch, y_batch in dataloader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")


In [ ]:
label_map = {
    "normal": 0,
    "tremor": 1,
    "tonic": 2,
    "postural": 3
}

In [ ]:
# Load dataset
dataset = MotionDataset("data.csv")

# Create model
model = SeizureLSTM(input_size=8, num_classes=4)  # assuming 4 classes: normal, tremor, tonic, posture

# Train
train(model, dataset)
